In [ ]:
# Set Jupyter to render directly to the screen
%matplotlib inline
import matplotlib as plt
# Import pandas and numpy for analysis, as well as package math
import pandas as pd
import numpy as np
import math as math

# Importing the Analysis Functions from the external Python Module

In [ ]:
# Import the functions to do the cointegration analysis. This requires that 'cointegration_analysis.py' is in the same 
# directory as this notebook.

from cointegration_analysis import estimate_long_run_short_run_relationships, engle_granger_two_step_cointegration_test

In [ ]:
# We can use the help() function in Python to bring up the docstrings. Alternatively, we could have used shift-tab while 
# highlighint the function name or looked in the cointegration_analysis.py file itself.

help(estimate_long_run_short_run_relationships)

In [ ]:
help(engle_granger_two_step_cointegration_test)

# Reading in the CSV file

In [ ]:
def read_data(filename):
    '''
    This function reads the .csv stored at the 'filename' location and returns a DataFrame
    with two levels of column names. The first level column contains the Stock Name and the 
    second contains the type of market data, e.g. bid/ask, price/volume.
    '''
    df = pd.read_csv(filename, index_col=0)
    df.columns = [df.columns.str[-2:], df.columns.str[:-3]]

    return df

In [ ]:
# Read the market data

filename = 'Pairs Trading.csv'
market_data = read_data(filename)

In [ ]:
# Get all the stock names into a list

stock_names = list(market_data.columns.get_level_values(0).unique())

print(stock_names)

# Some Examples

In [ ]:
# What is in the dataframe? (Display top 5 rows with the .head() DataFrame method.)

market_data.head()

In [ ]:
# Extracting the BidVolumes of AA. Note how the printed output is formatted differently 
# from the output printed for the whole DataFrame, to indicate that the single column 
# we extracted is a Series.

bid_volumes_AA = market_data['AA', 'BidVolume']
bid_volumes_AA.head()

In [ ]:
# Extracting the BidVolume at a specific time. This is just a number (float).

time = '2018-01-05 10:20:00'

bid_volume_AA_at_time = market_data.loc[time, ('AA', 'BidVolume')]
bid_volume_AA_at_time

In [ ]:
# Extracting a subset of observations, here the 1220th to the 1225th.

market_data.iloc[1220:1225]

In [ ]:
# Adding a new column based on a calculation of old columns
# NOTE: The new column gets added to the far right of the DataFrame

bid_ask_spread_AA = market_data['AA', 'AskPrice'] - market_data['AA', 'BidPrice']

market_data['AA', 'BidAskSpread'] = bid_ask_spread_AA

market_data.head()

In [ ]:
# The resulting DataFrame still looks a bit disordered, with the new AA column not alongside
# the other AA columns. We can alphabetically sort the column names to clean things up a bit.

market_data = market_data.sort_index(axis=1)
market_data.head()

In [ ]:
# If we want to iterate over each timestamp, we can easily do so. As practice, let's 
# calculate the maximum BidAskSpread of AA seen in the whole dataset.

max_spread_seen = 0
max_spread_seen_time = None

for time, mkt_data_at_time in market_data.iterrows():
    spread = mkt_data_at_time['AA', 'BidAskSpread'] 
    
    if spread > max_spread_seen:
        max_spread_seen = spread
        max_spread_seen_time = time

print(max_spread_seen_time, max_spread_seen)

In [ ]:
# Now let's do a comparison between different timestamps. We will calculate the maximum 
# price-increase of the AA BidPrice for the whole dataset.

max_price_increase = -999999
max_price_increase_seen_time = None

prev_time = None

for time, mkt_data_at_time in market_data.iterrows():
    if prev_time == None:
        # Skip the first observation, there is no previous bid price to compare to yet.
        prev_time = time
        continue
    
    previous_bid_price = market_data.loc[prev_time, ('AA', 'BidPrice')]  
    current_bid_price = mkt_data_at_time['AA', 'BidPrice']
       
    bid_price_increase = current_bid_price - previous_bid_price
    
    if bid_price_increase > max_price_increase:
        max_price_increase = bid_price_increase
        max_price_increase_seen_time = time
        
    # Update the previous time for the next iteration of the loop.
    prev_time = time

print(max_price_increase_seen_time, max_price_increase)

# Exercise

In [ ]:
# Now it's your turn. Start with plotting some of the values in the market_data DataFrame. Next, apply
# the analysis functions imported above. Finally design and backtest a trading algorithm exploiting
# the cointegration relationships found in the data.


# Analysis

In [ ]:
# Load and eplore dataset
filename = 'Pairs Trading.csv'
market_data = read_data(filename)

market_data.head()

In [ ]:
# Quick plot of bid and ask prices of 2 stocks
market_data['BB', 'BidPrice'].iloc[0:250].plot(color='teal', linestyle=':', label = "BB_BidPrice", legend = True)
market_data['BB', 'AskPrice'].iloc[0:250].plot(color='orange', linestyle=':', label = "BB_AskPrice", legend = True)
market_data['CC', 'BidPrice'].iloc[0:250].plot(color='teal', title='prices', figsize=(20, 3),  label = "CC_BidPrice", legend = True)
market_data['CC', 'AskPrice'].iloc[0:250].plot(color='orange', label = "CC_AskPrice", legend = True)


In [ ]:
# Quick plot of volume series
market_data['AA', 'BidVolume'].iloc[0:250].plot(color='teal', label = "AA_BidVolume", legend = True, figsize = (20,3))
market_data['AA', 'AskVolume'].iloc[0:250].plot(color='orange', label = "AA_AskVolume", legend = True)
market_data['DD', 'BidVolume'].iloc[0:250].plot(color='teal', linestyle=':', label = "DD_BidVolume", legend = True)
market_data['DD', 'AskVolume'].iloc[0:250].plot(color='orange', title = "Volume", linestyle=':', label = "DD_AskVolume", legend = True)



In [ ]:
# Add market mid prices columns

for stock in stock_names:
    market_data[stock,"MidPrice"] = (market_data[stock,'BidPrice'] + market_data[stock, 'AskPrice']) / 2.0
        
        
market_data = market_data.sort_index(axis=1)
market_data.head()        



In [ ]:
# Create new columns with log prices, because we want to work with log prices to calculate cointegration
for stock in stock_names:
    market_data[stock,"MidPriceLog"] = np.log(market_data[stock,'MidPrice'])
    market_data[stock,"AskPriceLog"] = np.log(market_data[stock,'AskPrice'])
    market_data[stock,"BidPriceLog"] = np.log(market_data[stock,'BidPrice'])      
        
market_data = market_data.sort_index(axis=1)
market_data.head()        


In [ ]:
# Create new columns with bid-ask spread, also with log prices and make it a percentage (Vidyamurthy page 83)

for stock in stock_names:
    market_data[stock,"BidAskSpread"] = (market_data[stock,'AskPrice'] - market_data[stock, 'BidPrice'])
    market_data[stock,"BidAskSpreadLogPerc"] = (market_data[stock,'AskPriceLog'] - market_data[stock, 'BidPriceLog']) / market_data[stock, 'AskPriceLog']
    
market_data = market_data.sort_index(axis=1)
market_data.head()     

In [ ]:
#Visualize mid prices to check if they are correct
market_data['BB', 'BidPrice'].iloc[0:250].plot(color='teal', linestyle=':', label = "BB_BidPrice", legend = True)
market_data['BB', 'AskPrice'].iloc[0:250].plot(color='orange', linestyle=':', label = "BB_AskPrice", legend = True)
market_data['BB', 'MidPrice'].iloc[0:250].plot(color='yellow', linestyle=':', label = "BB_MidPrice", legend = True)
market_data['CC', 'BidPrice'].iloc[0:250].plot(color='teal', title='MidPrices', figsize=(20, 3),  label = "CC_BidPrice", legend = True)
market_data['CC', 'AskPrice'].iloc[0:250].plot(color='orange', label = "CC_AskPrice", legend = True)
market_data['CC', 'MidPrice'].iloc[0:250].plot(color='yellow', label = "CC_MidPrice", legend = True)

In [ ]:
# Try analysis functions on BB and CC. Conclusion: not a good pair to trade (high p-value)
print(engle_granger_two_step_cointegration_test(market_data['BB','MidPriceLog'], market_data['CC', 'MidPriceLog']))
estimate_long_run_short_run_relationships(market_data['BB','MidPriceLog'], market_data['CC', 'MidPriceLog'])

In [ ]:
# Run the analysis functions for all pairs by creating a nested loop and storing them in dictionaries
stock_names_duplicate = list(market_data.columns.get_level_values(0).unique())

dict_pvalue = {}
dict_dstat = {}
dict_c = {}
dict_gamma = {}
dict_alpha = {}
dict_z = {}


for stock in stock_names:
    for stockduplicate in stock_names_duplicate:
        if stock != stockduplicate and (stockduplicate, stock) not in dict_pvalue:
            dict_pvalue[(stock, stockduplicate)] = engle_granger_two_step_cointegration_test(market_data[stock,'MidPriceLog'], market_data[stockduplicate, 'MidPriceLog'])[1]
            dict_dstat[(stock, stockduplicate)] = engle_granger_two_step_cointegration_test(market_data[stock,'MidPriceLog'], market_data[stockduplicate, 'MidPriceLog'])[0]
            dict_c[(stock, stockduplicate)] = estimate_long_run_short_run_relationships(market_data[stock,'MidPriceLog'], market_data[stockduplicate, 'MidPriceLog'])[0]
            dict_gamma[(stock, stockduplicate)] = estimate_long_run_short_run_relationships(market_data[stock,'MidPriceLog'], market_data[stockduplicate, 'MidPriceLog'])[1]
            dict_alpha[(stock, stockduplicate)] = estimate_long_run_short_run_relationships(market_data[stock,'MidPriceLog'], market_data[stockduplicate, 'MidPriceLog'])[2]
            dict_z[(stock, stockduplicate)] = estimate_long_run_short_run_relationships(market_data[stock,'MidPriceLog'], market_data[stockduplicate, 'MidPriceLog'])[3]

In [ ]:
# Create dataframes with pvalues and test statistics, as well as long run short run relationship
pvalues_df = pd.DataFrame.from_dict(dict_pvalue, orient='index')
pvalues_df.rename(columns = {0: 'p_value'}, inplace = True)
print(pvalues_df.head())

dstat_df = pd.DataFrame.from_dict(dict_dstat, orient='index')
dstat_df.rename(columns = {0: 'd_stat'}, inplace = True)
dstat_df.head()

c_df = pd.DataFrame.from_dict(dict_c, orient='index')
c_df.rename(columns = {0: 'c_value'}, inplace = True)
print(c_df.head())

gamma_df = pd.DataFrame.from_dict(dict_gamma, orient='index')
gamma_df.rename(columns = {0: 'gamma_value'}, inplace = True)
print(gamma_df.head())

alpha_df = pd.DataFrame.from_dict(dict_alpha, orient='index')
alpha_df.rename(columns = {0: 'alpha_value'}, inplace = True)
print(alpha_df.head())

z_df = pd.DataFrame.from_dict(dict_z, orient='index')
z_df.rename(columns = {0: 'z_value'}, inplace = True)
z_df.head()

In [ ]:
# Inspect z dataframe
error_df=z_df.T
error_df.head()

In [ ]:
# Sort pvalues (lower is better)
pvalues_df.sort_values(by = "p_value", inplace = True)
pvalues_df.head(10)

In [ ]:
# Sort dickey fuller statistic (more negative is better)
dstat_df.sort_values(by = "d_stat", inplace = True)
dstat_df.head(10)

In [ ]:
# Sort alpha values (higher absolute value is better), 
# these together with p-value and dickey fuller statistic determine the right pairs to trade
alpha_df.sort_values(by = "alpha_value", inplace = True)
alpha_df.head(10)

In [ ]:
#
c_df.sort_values(by = "c_value", inplace = True)
c_df.head(10)

In [ ]:
# Best pairs to trade according to p-value
# Conclusion: (FF,MM) (BB, JJ), (FF,NN), (BB, DD), (MM, NN), (DD, JJ), (DD, HH) are all below 0.0001

pvalues_df.head(10).plot(kind = "bar")

In [ ]:
# Best pairs to trade according to dstat
dstat_df.head(10).plot(kind = "bar")

In [ ]:
# Best pairs to trade according to alpha
alpha_df.head(10).plot(kind = "bar")

In [ ]:
# Worst pairs according to p-value
pvalues_df.tail(10).plot(kind = "bar")

In [ ]:
# Worst pairs to trade according to dstat
dstat_df.tail(10).plot(kind = "bar")

In [ ]:
# Create a copy as a fail safe
market_data_copy = market_data.copy()

In [ ]:
# Merge market data and the z (error correction term) dataframe
market_data = pd.merge(market_data,error_df,left_index=True, right_index=True )
market_data = market_data.sort_index(axis=1)
market_data.head() 

In [ ]:
# Visualize bid and ask volume

# Plot of volume series
market_data['FF', 'BidVolume'].iloc[0:250].plot(color='teal', label = "FF_BidVolume", legend = True, figsize = (20,6))
market_data['FF', 'AskVolume'].iloc[0:250].plot(color='orange', label = "FF_AskVolume", legend = True)
market_data['MM', 'BidVolume'].iloc[0:250].plot(color='teal', linestyle=':', label = "MM_BidVolume", legend = True)
market_data['MM', 'AskVolume'].iloc[0:250].plot(color='orange', title = "Volume", linestyle=':', label = "MM_AskVolume", legend = True)


In [ ]:
# Visualize bid and ask prices
market_data['FF', 'BidPrice'].plot(color='teal', label = "FF_BidPrice", legend = True, figsize = (20,6))
market_data['FF', 'AskPrice'].plot(color='orange', label = "FF_AskPrice", legend = True)
market_data['MM', 'BidPrice'].plot(color='teal', linestyle=':', label = "MM_BidPrice", legend = True)
market_data['MM', 'AskPrice'].plot(color='orange', title = "Bid and ask prices", linestyle=':', label = "MM_AskPrice", legend = True)

market_data['FF', 'BidPrice'].plot(color='teal', label = "FF_BidPrice", legend = True, figsize = (20,6))
market_data['FF', 'AskPrice'].plot(color='orange', label = "FF_AskPrice", legend = True)
market_data['MM', 'BidPrice'].plot(color='teal', linestyle=':', label = "MM_BidPrice", legend = True)
market_data['MM', 'AskPrice'].plot(color='orange', title = "Bid and ask prices", linestyle=':', label = "MM_AskPrice", legend = True)


In [ ]:
# Visualize log of mid prices (just like bid-ask graph, cointegration is easily spotted here)
market_data['FF', 'MidPriceLog'].plot(color='orange', label = "FF_MidPriceLog", legend = True, figsize = (20,6))
market_data['MM', 'MidPriceLog'].plot(color='orange', title = "MidPricesLog", linestyle=':', label = "MM_MidPriceLog", legend = True)

In [ ]:
# Visualize error correction term with lines at 2 * standard deviation (theoretical optimal z)

# Calculate standard deviation of Z, optimal threshold is 2 * standard deviation
#(paper: Pairs trading: optimal thresholds and profitability) -> trade below 2 standard deviation would be a trade of poor quality

optimal_z = market_data['FF', 'MM'].std() * 2
optimal_z

market_data.plot(y = [('FF', 'MM')], title = 'Error correction term FF,MM', figsize = (20,6))
plt.pyplot.axhline(y = optimal_z, color = 'r')
plt.pyplot.axhline(y = -optimal_z, color = 'g')

In [ ]:
# Get maximum and minimum values of error correction term
print(market_data['FF', 'MM'].idxmax())
print(max(market_data['FF', 'MM']))
print(market_data['FF', 'MM'].idxmin())
print(min(market_data['FF', 'MM']))

In [ ]:
# Inspect FF,MM pair. Conclusion: if error correction term is high, sell the spread -> sell FF, buy MM
# if the error correction term is low, buy the spread -> buy FF, sell MM
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

time_max = '2018-02-01 16:10:00'

max_MidPrice_FF_at_time = market_data.loc[time_max, ('FF', 'MidPrice')]
max_MidPrice_MM_at_time = market_data.loc[time_max, ('MM', 'MidPrice')]
print('Max price FF is',max_MidPrice_FF_at_time, 'Max price MM is', max_MidPrice_MM_at_time)

time_min = '2018-02-20 02:50:00'

min_MidPrice_FF_at_time = market_data.loc[time_min, ('FF', 'MidPrice')]
min_MidPrice_MM_at_time = market_data.loc[time_min, ('MM', 'MidPrice')]
print('Min price FF is', min_MidPrice_FF_at_time, 'Min price MM is', min_MidPrice_MM_at_time)

In [ ]:
# Make a dataframe with the relationship coefficients (c, gamma, alpha, p-value, dickey fuller statistic)
rel_coeff = pd.merge(c_df, gamma_df,left_index=True, right_index=True )
rel_coeff = pd.merge(rel_coeff, alpha_df,left_index=True, right_index=True )
rel_coeff = pd.merge(rel_coeff, pvalues_df,left_index=True, right_index=True )
rel_coeff = pd.merge(rel_coeff, dstat_df,left_index=True, right_index=True )
rel_coeff = rel_coeff.copy()
rel_coeff.head()


In [ ]:
# Determine optimal pairs basede on p-value, alpha and dickey fuller statistic
# We want a p_value below 0.0001, an alpha below 0.002 and a dickey fuller statistic below 6

pairs_traded = rel_coeff.loc[rel_coeff["p_value"] < 0.0001].loc[abs(rel_coeff["alpha_value"]) > 0.002].loc[abs(rel_coeff["d_stat"]) > 6]
pairs = list(pairs_traded.T.columns.get_level_values(0).unique())

for pair in pairs:
    print(rel_coeff.loc[[pair],:])
    
pairs

In [ ]:
# Create a list with the individual stocks in our optimal pairs
stocks=[]
for pair in pairs:
    for stock in pair:
        if stock not in stocks:
            stocks.append(stock)
            
stocks

In [ ]:
# Create a function that checks if a number is whole
def is_whole(n):
    return n % 1 == 0

print(is_whole(4.5), is_whole(10))

# Algorithm 

In [ ]:
### Algorithm ### 
limit = 100

# Positions dataframe, the initial values are all 0.          
positions = pd.DataFrame(data=0, 
                         index=market_data.index,
                         columns= stocks)       

for time, mkt_data_at_time in market_data.iterrows():
    if prev_time == None:
        # This skips the first observation, we don't want to take a position yet
        prev_time = time
        continue


    # Loop over each pair and consecutive timestamp in the DataFrame and make a trading decision at each timestamp, 
    # dynamically keeping track of the previous timestamp to easily refer to the previous positions.
    # Because our algorithm only works for the pairs individually but not all together, this output is for one pair
    for pair in pairs:
        stock1=pair[0]
        stock2=pair[1]

        # Trade ratio is 1:(gamma * y_t-1 / x_t-1)
        gamma = rel_coeff.loc[[pair], 'gamma_value']
        gamma = gamma.iloc[0]
        previous_price_stock1 = market_data.loc[prev_time, (stock1, 'MidPrice')]
        previous_price_stock2 = market_data.loc[prev_time, (stock2, 'MidPrice')]
        
        trade_ratio = gamma* previous_price_stock1/previous_price_stock2
        optimal_z = market_data[stock1, stock2].std() * 2
        
        # If the error correction term is higher than 0.005:
        if mkt_data_at_time[stock1, stock2] > optimal_z:

            # Check if trade is feasible with trading slippage (Vidyamurthy book, page 83)
            # If trading slippage < optimal z, the trade is feasible
            if (mkt_data_at_time[stock1, 'BidAskSpreadLogPerc'] + trade_ratio * mkt_data_at_time[stock2, 'BidAskSpreadLogPerc']) < optimal_z:

            # Then sell the spread -> sell FF and buy MM.
            # But never trade more than the volume available on either side.
                volume_available = min(mkt_data_at_time[stock1, 'BidVolume'], 
                                       mkt_data_at_time[stock2, 'AskVolume'])

                previous_position_stock1 = positions.loc[prev_time, stock1]
                previous_position_stock2 = positions.loc[prev_time, stock2]
                
            # Sell stock1 and buy stock 2, so for stock1 the ideal position is -volume available and vice versa
                ideal_position_stock1 = previous_position_stock1 - volume_available 
                ideal_position_stock2 = previous_position_stock2 + volume_available 

            # We would like to trade up to ideal_position, but 
            # if that is more than the limit of one of the stocks, we will trade up to the limit instead
                limited_position_stock1 = max(ideal_position_stock1, -limit)
                limited_position_stock2 = min(ideal_position_stock2, limit)
                
            # To stay within our limit, we calculate the lowest absolute limited position of the pair,
            # using the previous position and the limited difference between the previous positions
            # and the limited positions
                limited_difference_stock1 = limited_position_stock1 - previous_position_stock1
                limited_difference_stock2 = limited_position_stock2 - previous_position_stock2
                
                limited_position = min(abs(limited_difference_stock1), abs(limited_difference_stock2))

    # Execute the trade
            # If the trade ratio > 1, we need to trade less of stock1 to stay within our limit. So, we divide
            # the initial position by the trade ratio to get a smaller number of stocks. We also use math.floor to 
            # get whole values (we can only trade whole units). 
                if  trade_ratio > 1:
                    stock1P = math.floor(previous_position_stock1-limited_position/trade_ratio)
                    stock2P = math.floor(previous_position_stock2+limited_position)
                    
                # Since we sell stock1, the new position should be lower than the previous position
                    if stock1P < positions.loc[prev_time, stock1]:
                        positions.loc[time, stock1] = stock1P
                        positions.loc[time, stock2] = stock2P
                # If this is not the case, we keep the previous position
                    else:
                        positions.loc[time, stock1] = positions.loc[prev_time, stock1]
                        positions.loc[time, stock2] = positions.loc[prev_time, stock2]
            # If the trade ratio < 1, we can trade at the limited position. So, the position of stock1 is 
            # previous position -limited position (sell) and the position of stock2 is
            # previous position + limited position (buy) * trade_ratio
            # We also use math.floor to get whole values (we can only trade whole units).                       
                else:
                    stock1P = math.floor(previous_position_stock1 - limited_position)
                    stock2P = math.floor(previous_position_stock2 + limited_position * trade_ratio)
                    if stock1P < positions.loc[prev_time, stock1]:
                        positions.loc[time, stock1] = stock1P
                        positions.loc[time, stock2] = stock2P
                    else:
                        positions.loc[time, stock1] = positions.loc[prev_time, stock1]
                        positions.loc[time, stock2] = positions.loc[prev_time, stock2]

        # Else buy the spread -> buy FF and sell MM
        elif mkt_data_at_time[stock1, stock2] < -optimal_z:

            # Check if trade is feasible with trading slippage (Vidyamurthy book, page 83)
            # If trading slippage < absolute value of optimal z, the trade is feasible
            if (mkt_data_at_time[stock1, 'BidAskSpreadLogPerc'] + trade_ratio * mkt_data_at_time[stock2, 'BidAskSpreadLogPerc']) < abs(optimal_z):

                # But never trade more than the volume available on either side.
                volume_available = min(mkt_data_at_time[stock1, 'AskVolume'], 
                                       mkt_data_at_time[stock2, 'BidVolume'])

                previous_position_stock1 = positions.loc[prev_time, stock1]
                previous_position_stock2 = positions.loc[prev_time, stock2]
                
            # Sell stock1 and buy stock 2, so for stock1 the ideal position is -volume available and vice versa
                ideal_position_stock1 = previous_position_stock1 + volume_available 
                ideal_position_stock2 = previous_position_stock2 - volume_available 

            # We would like to trade up to ideal_position, but 
            # if that is more than the limit of one of the stocks, we will trade up to the limit instead
                limited_position_stock1 = min(ideal_position_stock1, limit)
                limited_position_stock2 = max(ideal_position_stock2, -limit)
                
            # To stay within our limit, we calculate the lowest absolute limited position of the pair,
            # using the previous position and the limited difference between the previous positions
            # and the limited positions                
                limited_difference_stock1 = limited_position_stock1 - previous_position_stock1
                limited_difference_stock2 = limited_position_stock2 - previous_position_stock2
                
                limited_position = min(abs(limited_difference_stock1), abs(limited_difference_stock2))

        #Execute the trade
             # If the trade ratio > 1, we need to trade less of stock1 to stay within our limit. So, we divide
             # the initial position by the trade ratio to get a smaller number of stocks. We also use math.floor to 
             # get whole values (we can only trade whole units).               
                if trade_ratio > 1: 
                    stock1P = math.floor(previous_position_stock1+limited_position/trade_ratio)
                    stock2P = math.floor(previous_position_stock2-limited_position)
                    
                # Since we buy stock1, the new position should be higher than the previous position                    
                    if stock1P > positions.loc[prev_time, stock1]:
                        positions.loc[time, stock1] = stock1P
                        positions.loc[time, stock2] = stock2P
                # If this is not the case, we keep the previous position                        
                    else:
                        positions.loc[time, stock1] = positions.loc[prev_time, stock1]
                        positions.loc[time, stock2] = positions.loc[prev_time, stock2]
            # If the trade ratio < 1, we can trade at the limited position. So, the position of stock1 is 
            # previous position +limited position (bu) and the position of stock2 is
            # previous position - limited position (sell) * trade_ratio
            # We also use math.floor to get whole values (we can only trade whole units).                          
                else:
                    stock1P = math.floor(previous_position_stock1 + limited_position)
                    stock2P = math.floor(previous_position_stock2 - limited_position * trade_ratio)

                    if stock1P > positions.loc[prev_time, stock1]:
                        positions.loc[time, stock1] = stock1P
                        positions.loc[time, stock2] = stock2P
                    else:
                        positions.loc[time, stock1] = positions.loc[prev_time, stock1]
                        positions.loc[time, stock2] = positions.loc[prev_time, stock2]

        # Or finally, if the error correction term is not above or below the optimal z
        else:
            # Hold the previous positions, no trades
            positions.loc[time, stock1] = positions.loc[prev_time, stock1]
            positions.loc[time, stock2] = positions.loc[prev_time, stock2]

        # In the next iteration of the loop, the previous time will be what is now the current time
        prev_time = time
    

In [ ]:
# Inspect positions for each stock at timestamps
positions.tail()

In [ ]:
# Visualize positions over time
positions.plot(figsize=(20, 3))

In [ ]:
# Traded lots, assume we trade 0 in the first period
trades = positions.diff().fillna(0)

# Again start with an empty DataFrame
pnl_trades = pd.DataFrame(index = trades.index, columns = stocks)


# Calculate total PnL made from trading (we bought against ask price, and sold against bid price).
lots_bought = np.maximum(trades, 0)
lots_sold = -np.minimum(trades, 0)

for stock in stocks:
    pnl_trades[stock] = lots_sold[stock] * market_data[stock, 'BidPrice'] - lots_bought[stock] * market_data[stock, 'AskPrice']

pnl_trades_total = pnl_trades.iloc[:,:].sum(axis=1)
pnl_trades_cumulative = pnl_trades_total.cumsum()

In [ ]:
# Evaluating the position at current midprice per stock
position_valuation = pd.DataFrame(data = 0, index = trades.index, columns = stocks)

for stock in stocks:
    position_valuation[stock] = market_data[stock, 'MidPrice'] * positions[stock]

position_valuation_total = position_valuation.iloc[:,:].sum(axis=1)

position_valuation_total.tail()

In [ ]:
# Total pnl from trades and position
pnl_total = pnl_trades_cumulative + position_valuation_total

# Plot it
pnl_trades_total.plot(figsize=(20, 3))

In [ ]:
# Visualize cumulative profit and loss
pnl_trades_cumulative.plot(figsize=(20, 3))

In [ ]:
# Visualize total pnl

pnl_total.plot(figsize=(20, 3), title='PnL for whole dataset')

In [ ]:
pnl_total.plot(label='Total PnL', 
                            figsize=(20, 3), 
                            title='Split out PnLs', 
                            legend=True)

pnl_trades_cumulative.plot(label='Cumulative Trades', 
                                        legend=True)

position_valuation_total.plot(label='Position Valuation', 
                                           legend=True)

In [ ]:
# Create output dataframe with positions and the PnL
output = pd.concat((positions, pnl_total), axis=1)
output.columns.values[[-1]] = ['Profit-and-Loss']
output.tail()

In [ ]:
# Create CSV output file 
output.to_csv("PnL.csv")

In [ ]:
# For loop to check if (trade ratio * some unit of stocks, up until maximum volume) is a whole number that we can trade,
# so that we preserve our trade ratio
# the optimal FF and MM will be 41 and 46, since these are whole numbers and close to our trade ratio of 1.122525
# (46/41) = 1.122
trade_ratio = 1.122525
limited_position = 50

if (trade_ratio * limited_position) > limited_position:
    FF = math.floor(-limited_position / trade_ratio)
    MM = math.floor(limited_position / trade_ratio) * trade_ratio
    for i in reversed(range(1, abs(FF))):
        if is_whole((i * trade_ratio)): 
            FF = i
            MM = i * trade_ratio
            break
        elif is_whole(round(i * trade_ratio, 1)):
            FF = i
            MM = round(i * trade_ratio, 1)
            break
        elif is_whole(i * round(trade_ratio, 1)):
            FF = i
            MM = (i * round(trade_ratio, 1))
            break
        else:
            FF = 0
            MM = 0
    print("If we go from" ,limited_position, "to 1", " the whole number", FF,
          "times", trade_ratio, "is closest to", MM)  

In [ ]:
# We tried to put it in this function, but it did not work in the algorithm
def trade_and_preserve_ratio(FF,trade_ratio):
    for i in reversed(range(1, abs(FF))):
        if is_whole((i * trade_ratio)): 
            trade_and_preserve_ratio.FF = i
            trade_and_preserve_ratio.MM = i * trade_ratio
            break
        elif is_whole(round(i * trade_ratio, 1)):
            trade_and_preserve_ratio.FF = i
            trade_and_preserve_ratio.MM = round(i * trade_ratio, 1)
            break
        elif is_whole(i * round(trade_ratio, 1)):
            trade_and_preserve_ratio.FF = i
            trade_and_preserve_ratio.MM = i * round(trade_ratio, 1)
            break